In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Transfer Learning with TensorFlow: Feature Extraction**

To improve models we can use several tenchniques like
- Adding More Layers
- Changing the Learning Rate
- Adjusting the Number of Neurons Per Layer

However, instead of above we can use **Transfer Learning**.
- **Transfer Learning** is taking the patterns (also called weights) from another model and applying it on new problem.

Two main benefits of using Transfer Learning.
- Leverage existing Neural Network Architecture proven to work on problems similar to our own
- Using **Already Learned Patterns** on similar data to our own

So, instead of hand-crafting our own **Neural Network Architecture or building them from scratch** we can utilize models which have worked for others.

We can take the patterns a model has learned from datasets such as **ImageNet** and use it as a foundational model.

**What we will Learn**
- Use a smaller dataset to experiment faster (10% of training samples of 10 classes of food)
- Build a Transfer Learning Feature Extraction model using **TensorFlow Hub**
- Introduce a TesnorBoard Callback to track model training results

**Transfer Learning with TensorFlow Hub: Getting great results with only 10% of data**

- **TensorFlow Hub:-** is a repository for existing model components. You can import and use a **Fully Trained Model** using a *URL*

Using the **Pre-trained Models** we can get the results of a fully trained model with only 10% of data.

**Transfer Learning often allows you to get great results with less data**


Let's download 10% of training data from **10_food_classes** dataset and use it to train a food image classifier on it.















In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-09-22 08:56:38--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.188.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   208MB/s    in 0.8s    

2025-09-22 08:56:39 (208 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

**Creating Data Loaders (Preparing the Data)**

- Create the **ImageDataGenerator** class using the **flow_from_directory** method to load in our images.

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training Images")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode="categorical"
                                                          )


print("Testing Images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical"
                                             )



Training Images
Found 750 images belonging to 10 classes.
Testing Images
Found 2500 images belonging to 10 classes.


**Setting Up Callbacks (Things to Run While our Model Trains)**

**Callbacks** are extra functionality that you can add to your models to be performed during or after training. Some of the most important callbacks are

- **Experiment Tracking with TensorBoard:-** Log the performance of multiple models and then view and compare these models in a visual way on **TensorBoard**. **TensorBoard** is a dashboard for inspecting **Neural Network Parameters**

- **Model CheckPointing:-** Save your model as you train so that you can stop training if needed and continue off where you left. It is helpful if training takes a long time and cannot be done in one sitting.

- **Early Stopping:-** Leave your model training for a arbitary amount of time and have it stop training automatically when it ceases to improve. It is helpful when you have a large dataset and do not know how long training will take.

- The TensorBoard Callback can be accessed using
**tf.keras.callbacks.TensorBoard()**. - The main function of this is saving model's training performance metrics to a specified **log_dir**.

- By default, logs are recorded every epoch using the **update_freq='epoch'** parameter. This is a good default but can slow down **Model Training.**





In [6]:
# Creating a tensorboard callback

import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


- We will save the **Model** to a directory [dir_name] / [experiment_name] / [current_timestamp] where
- **dir_name:-** is the overall logs directory
- **experiment_name:-** is the particular experiment
- **current_timestamp:-** is the time the experiment started based on Python time *datatime.datetime().now()

**Creating Models Using TensorFlow Hub**

- In past we used to create models from scratch
- In here, majority of our model's layers are going to come from **TensorFlow Hub**

We will use two models from **TensorFlow Hub**

- 1.**ResNetV2** - a state of the art computer vision model architecture from 2016
- 2.**EfficientNet**- a state of the art computer vision model from 2019

By the state of art we mean that majority of our modeles have achieved the lowest error rate on **ImageNet (ILSVRC-2012-CLS)**, the gold standard of computer vision benchmarks.

Steps for finding models on **TensorFlow Hub**
1. Got to **tfhub.dev**
2. Select the problem domain like **Image**
3. Remove all **Problem Domain** filters except for the one you are working on
4. You will see a list of models, select the one you want to use

**I see many models, then which one is to be used**

- You can find a list of state of the art models on **paperswithcode.com**, a resource for collecting the latest in deep learning paper results

- Since, our target is **Image Classification** so we would use the model that performed best on **ImageNet.**

- On **tfhub.dev** you will find various architectures like **EfficientNetB4** which is better than **EfficientNetB0** but larger models take alot of time to compute.


We will use feature vectors URLs of two common computer vision architectures, **EfficientNetBO (2019)** and **ResNetV250 (2016)**

Why we select only **Feature Vectors**.
Because Transfer Learning come into play as **Feature Extraction** and **Fine Tuning**

1. In **transfer Learning** we take a pre-trained models as it is and apply it to our task without changes. For example, if your model is trained on **ImageNet** dataset that contains **1000** different classes of images. So, if we pass a single image to this model it will produce **1000** different outputs. It can be useful if we want to classify **1000** images.

2. **Feature Extraction Transfer Learning** is a process where you take the underlying patterns (also called weights) a **pretrained Model** has learned and adjust its output to be more suited to your problem.
For example, If your model had 236 different layers (EfficientNetBO has 236 layers) and the top layer outputs **1000** classes because it was pretrained on **ImageNet.**To adjust it to your problem we might remove the top layer and replace it with our own having the right number of classes. The most important part here is that **only the top few layers become trainable, the rest remain frozen.**So, the underlying patterns remain in the rest of layers and we can utilize it for our problem.

3. **Fine-Tuning Transfer Learning:-** is when you take the underlying patterns (also called weights) of a pre-trained model and adjust them to your problem. This means **training some, many or all layers** in pretrained model. This is applicable in scenarios where you have relatively large dataset and **your data is slightly different** from the original data on which the model was trained.


A common practice is to **Freeze** all the learned patterns in bottom layers of a **pretrained model** so that they become **un-trainable**. Then, the top 2-3 layers of the **pre-trained** model can adjust its output to our customer data (**feature extraction**).
As you have trained the **top layers** you can gradually **unfreeze** more and more layers and run the training process on your own data to further **fine-tune**it.

- **Lower Layers** in a computer vision model learns **large features**. In a cat and dog classification they might learn the **outline of legs** while the layers closer to the output might learn **shape of the teeth.**


So, in **Feature Extraction** only the top 2-3 layers change but in **Fine Tuning Model** many or all of the original model get changed.





In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [8]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"


- The **URLs** are link to a saved **pretrained models** on **TensorFlow Hub**.
- When we use it in our model, the model will automatically downloaded for us to use
- We will use **KerasLayer()** inside the TensorFlow Hub library.

- The function below helps in creating **Model**.
- Our first model will be **ResNetV250** architecture as our feature extraction layer
- Once our model is instantiated, we will compile it using **categorical_crossentropy** as our loss function, **Adam Optimizer** and **Accuracy** as metric.

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

# Define constants
IMAGE_SHAPE = (224, 224)
NUM_CLASSES = 10  # Replace with the actual number of classes in your dataset
RESNET_URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Custom layer to wrap hub.KerasLayer
class HubLayer(tf.keras.layers.Layer):
    def __init__(self, model_url, **kwargs):
        super(HubLayer, self).__init__(**kwargs)
        self.hub_layer = hub.KerasLayer(model_url, trainable=False)

    def call(self, inputs):
        return self.hub_layer(inputs)

def create_model_tf_hub_fixed(model_url, num_classes=NUM_CLASSES):
    # Define input layer
    inputs = keras.Input(shape=IMAGE_SHAPE + (3,))

    # Apply custom Hub layer
    x = HubLayer(model_url)(inputs)

    # Add dropout and dense layers
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = keras.Model(inputs, outputs)

    return model

# Clear Keras session
tf.keras.backend.clear_session()

# Disable mixed precision
tf.keras.mixed_precision.set_global_policy('float32')

# Create and compile the model
resnet_model = create_model_tf_hub_fixed(RESNET_URL, num_classes=NUM_CLASSES)
resnet_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

# Display model summary
resnet_model.summary()




Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hub_layer (HubLayer)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,490 (80.04 KB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 0 (0.00 B)

**Adding the CallBack**
- A tensorflow **CallBack** adds extra functionality by virtue of which we can track the performance of our model on TensorBoard
- The **CallBack** is added in the **fit** method

In [ ]:
resnet_history = resnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                           experiment_name="resnet50V2")]
)


Saving TensorBoard log files to: tensorflow_hub/resnet50V2/20250922-085648


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 7/24 ━━━━━━━━━━━━━━━━━━━━ 15:20 54s/step - accuracy: 0.1112 - loss: 2.7934

**Transfer Learning with TensorFlow Part 2: Fine-Tuning**

- In the section above we saw how could we leverage **Feature Extraction Transfer Learning** to get far better results than building our own models
- In **Fine Tuning Transfer Learning** the pre-trained weights from another model are **unfrozen and tweaked** during training to better suit your own data
- In the **Feature Extraction Transfer Learning** we train the top **1-3 layers** of a pre-trained model with your own data
- In the **Fine-tuning transfer learning** we might train 1 to 3 or more layers of a pre-trained model

**Main Difference between Feature Extraction Transfer Learning and Fine Tuning Transfer Learning**
- **Feature Extraction Transfer Learning:** You only train only top layers
- **Fine Tuning Transfer Learning:** Not only top layers but some other layers beneath can be unfrozen for training.

**Path Ahead**

- Introducing Fine-Tuning: A type of transfer learning to modify a pre-trained model to more suited to your data
- Using **Keras Functional API**, a different way to build models in keras
- Using smaller dataset to experiment faster
- **Data Augmentation:** How to making training diverse without adding any new data

In [ ]:
### We create a bunch of helpful functions throughout the course.
### Storing them here so they're easily accessible.

import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.

    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results